In [48]:
import numpy as np
from ucimlrepo import fetch_ucirepo
from pandas import concat
import scipy.stats as stats
from sklearn import precision

Instrucciones.

Diseña una clase que implemente un clasificador bayesiano ingenuo (sin usar funciones de sklearn)


La clase debe construirse recibiendo un único parámetro que indica si la distribución de variables continuas se estimará usando la normal o KDE.


Un metodo ".fit()" que debe recibir como parámetros de entrada:


Los datos de entrenamiento X, y. Donde "X" guarda los valores de las variables y "y" la clase a la que pertenece cada dato.


Un diccionario que indique las variables (indices o nombres de columnas) que son "continuas", "enteras" y "categóricas".


El método ".fit()" debe calcular:
    Las probabilidades a priori de cada clase en Y. Es decir, P(Y=y_i)

    Las funciones de masa o densidad de probabilidad condicionales p(X_j = x | Y = y_i), para cada j=1,...,L y y_i para i=1,...,M, donde L es la dimensión de los datos y M el número de clases.


Para las variables categóricas debe usar la distribución Bernoulli (o en general distribución categórica).


Para las variables enteras calcular la media y estimar el pdf de una distribución Poisson.


Para las variables continuas calcular la media y la varianza de los datos para estimar el pdf de la normal, o estimar el KDE, de acuerdo con el parámetro de la clase.


El método ".predict()" el cual debe:
    Recibir un vector X_pred que debe tener el mismo número de columnas que X.
    Calcular el clasificador bayesiano ingenuo \PI_{j=1}^L p(X_j=x_j | Y=y_i)P(Y=y_i) para cada clase i=1,...,M y devolver el valor de y_i que lo maximiza.


Prueba tu clasificador usando los datos de Heart Disease.


Divide tus datos de entrenamiento y prueba de forma aleatoria. Usa proporciones 70-30 u 80-20.


Elige entre 5 y 7 variables para tu clasificación. Debe incluir por lo menos una categórica, una entera y una continua.


Calcula la exactitud global de tu clasificación.




Recomendaciones:

Para la normal se sugiere utilizar el método ".pdf()" de la función "scipy.stats.norm"


Para la poisson se sugiere utilizar el método ".pdf()" de la función "scipy.stats.poisson"


Para el KDE se sugiere utilizar el método ".evaluate()" de la función "scipy.stats.gaussian_kde"

In [49]:
class Classification:
    def __init__(self, classification_data):
        self.__vector = self.create_vector(classification_data)
        self.__classes = self.create_classes(classification_data)
        self.__matrix = self.create_matrix()
        self.__set = self.create_set()
        self.__class_dict = self.create_data_types()
        self.__elements_number = len(self.__vector)
    
    def create_vector(self, classification_data):
        # En lugar de eliminar los índices, que es lo que causa el error,
        # simplemente selecciona todas las columnas excepto 'num'
        # o extrae directamente los valores que necesitas
        if 'num' in classification_data.columns:
            # Si deseas excluir la columna 'num'
            __vector = classification_data.drop('num', axis=1).values
        else:
            # Si solo quieres obtener todos los valores
            __vector = classification_data.values
        return __vector
    
    def create_classes(self, classification_data):
        # La línea actual intentaría obtener classification_data['num'].values,
        # lo que probablemente no es lo que quieres
        # Corrige para acceder correctamente a la columna 'num'
        if 'num' in classification_data.columns:
            __classes = classification_data['num'].values
        else:
            # Manejo de error si no hay columna 'num'
            raise ValueError("La columna 'num' no existe en los datos")
        return __classes

    def create_matrix(self):
        # Obtener las clases únicas
        index = self.get_classes()
        matrix = []

        # Para cada elemento del vector, creamos una fila en la matriz
        for i in range(len(self.__vector)):
            row = []
            # Si el elemento del vector tiene la misma clase que el índice actual,
            # asignamos 1, de lo contrario 0
            for j in index:
                # Asumiendo que self.__vector[i] es un elemento individual y no un array
                # Y que self.__classes[i] contiene la clase a la que pertenece ese elemento
                row.append(1 if self.__classes[i] == j else 0)
            matrix.append(row)

        return matrix
        
    def create_set(self):
        # Crear el diccionario usando las clases únicas como claves
        set_rep = {i: set() for i in self.get_classes()}

        # Para cada elemento, agregar su índice al conjunto correspondiente a su clase
        for i in range(len(self.__vector)):
            # Usar la clase del elemento como clave, en lugar del vector completo
            class_value = self.__classes[i]  # Usamos la clase en lugar del vector
            set_rep[class_value].add(i)

        return set_rep

    def create_data_types(self):
        __class_dict = {'num': 'categorical',
                      'age': 'integer',
                      'sex': 'categorical',
                      'cp': 'categorical',
                      'trestbps': 'integer',
                      'chol': 'integer',
                      'fbs': 'categorical',
                      'restecg': 'categorical',
                      'thalach': 'integer',
                      'exang': 'categorical',
                      'oldpeak': 'float',
                      'slope': 'categorical',
                      'ca': 'integer',
                      'thal': 'integer'}
        return __class_dict

    def get_vector(self):
        return self.__vector
    
    def get_matrix(self):
        return self.__matrix
        
    def get_set(self):
        return self.__set
    
    def get_elements_number(self):
        return self.__elements_number
    
    def get_classes(self):
        return self.__classes

    def get_classes_dict(self):
        return self.__class_dict
    
    def get_class_probability(self, class_id):
        # If the class ID is not in the classifier, the probability is 0,
        # otherwise, the probability is computed as the proportion of
        # instances
        
        if class_id not in self.get_classes():
            return 0
        
        return len(self.get_set()[class_id]) / self.get_elements_number()


In [50]:
class BayesianClassifier():
    def __init__(self, Classification):
        self.__vector = Classification.get_vector()
        self.__classes = Classification.get_classes()
        self.__matrix = Classification.get_matrix()
        self.__data_type = Classification.get_classes_dict()
        self.__p_apriori = self.fit()

    def fit(self):
        """
        ***Description***

        :params:
        attribute_data : Pandas.DataFrame
        A DataFrame containing the attributes of the patterns that will be used to learn the classes. Will have l columns and n rows.

        objective_data : Pandas.DataFrame
        A DataFrame containing a single column that indicates the class of the patterns. Must have n rows.

        data_type : dict
        A dictionary whose length has to be the same as the attributes vector of the training dataframe (l entries). It indicates wether the data of a column is "continous", "integer" or "categoric".
        """
        # Obtener las clases únicas
        import numpy as np
        classes = np.unique(self.__classes)

        # Calcular la probabilidad a priori para cada clase
        rows = len(self.__classes)
        ocurrencias = np.array([np.sum(self.__classes == element) for element in classes])

        # Devolver probabilidades a priori
        return dict(zip(classes, ocurrencias / rows))

    def bernoulli(self, vector, x):
        import numpy as np
        vector_list = vector.tolist() if isinstance(vector, np.ndarray) else list(vector)
        rows = len(vector_list)
        unique_values = sorted(set(vector_list))
        ocurrencias = np.array([vector_list.count(element) for element in unique_values])
        p = ocurrencias / rows

        try:
            idx = unique_values.index(x)
            return p[idx]
        except ValueError:
            return 0  # Value not found in training data

    def poisson(self, vector, x):
        import numpy as np
        from scipy import stats
        vector_array = np.array(vector)
        lmbda = np.mean(vector_array)
        p = stats.poisson.pmf(x, lmbda)
        return p

    def normal(self, vector, x):
        import numpy as np
        from scipy import stats
        vector_array = np.array(vector)
        mu = np.mean(vector_array)
        sigma = np.std(vector_array)
        if sigma == 0:
            return 1 if x == mu else 0
        p = stats.norm.pdf(x, mu, sigma)
        return p

    def probability(self, vector, column_idx, x):
        """
        Calculate probability using appropriate distribution based on data type
        """
        # Get ordered list of feature names (excluding 'num' which is the class column)
        feature_names = [key for key in self.__data_type.keys() if key != 'num']

        # If column_idx is out of range, default to 'integer' type
        if column_idx >= len(feature_names):
            data_type = 'integer'
        else:
            # Get the column name for this index
            column_name = feature_names[column_idx]
            data_type = self.__data_type[column_name]

        # Calculate probability based on data type
        if data_type == 'continuous' or data_type == 'float':
            p = self.normal(vector, x)
        elif data_type == 'categorical':
            p = self.bernoulli(vector, x)
        else:
            p = self.poisson(vector, x)

        return p

    def conditional_probability(self, class_value, feature_idx, feature_value):
        """
        Calculate P(feature_value | class_value)
        """
        import numpy as np
        # Filter matrix for the specified class
        # Get indices where classes match the class_value
        class_indices = np.where(self.__classes == class_value)[0]

        if len(class_indices) == 0:
            return 0

        # Use these indices to filter the matrix
        class_matrix = np.array([self.__matrix[i] for i in class_indices])

        # If class_matrix is empty, return 0
        if len(class_matrix) == 0:
            return 0

        # Extract the feature column
        feature_column = [row[feature_idx] if isinstance(row, (list, np.ndarray)) and feature_idx < len(row)
                         else row for row in class_matrix]

        return self.probability(feature_column, feature_idx, feature_value)

    def predict(self, x_pred):
        """
        Predict the class for a new observation using Naive Bayes

        Arguments:
            x_pred: list or array with feature values in the same order as training data

        Returns:
            The predicted class
        """
        import numpy as np
        if isinstance(x_pred, list):
            x_pred = np.array(x_pred)

        probs = {}
        # Use unique class values from the training data instead of data_type keys
        unique_classes = np.unique(self.__classes)

        for class_value in unique_classes:
            # Start with prior probability P(class)
            prob = self.__p_apriori[class_value]

            # Multiply by each conditional probability P(feature|class)
            for i, feature_value in enumerate(x_pred):
                prob *= self.conditional_probability(class_value, i, feature_value)

            probs[class_value] = prob

        # Return the class with highest probability
        return max(probs, key=probs.get)

In [51]:
# fetch dataset

heart_disease = fetch_ucirepo(id = 45) 
  
# data (as pandas dataframes)

x = heart_disease.data.features 
y = heart_disease.data.targets 
  
data = concat([y, x], axis = 1)
data

,num,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0
1,2,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0
2,1,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0
3,0,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0
4,0,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,1,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0
299,2,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0
300,3,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0
301,1,57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0


In [52]:
# Create train-test splits
split = 0.8

# Dividir el DataFrame aleatoriamente
train_data = data.sample(n=int(split*len(data)), random_state=42)
test_data = data.drop(train_data.index)

In [53]:
X_test = test_data.drop("num", axis = 1).to_dict('records')
y_test = test_data["num"].to_dict()

In [54]:
# Create our class iterations
classification = Classification(train_data)
classifier = BayesianClassifier(classification)

In [55]:
# Predict over the top element
classifier.predict([63, 1, 1, 145, 233, 1, 2, 150, 0, 2.3, 3, 0.0, 6.0])

0

In [56]:
# Create predictions over the entire test dataset
y_preds = []

for x in X_test:
    x_vals = list(x.values())
    y = classifier.predict(x_vals)
    y_preds.append(y)

In [57]:
# We now measure the global precision

correct_preds = sum(1 for pred, real in zip(y_preds, y_test.values()) if pred == real)
precision = correct_preds / len(X_test)

print("Global precision:", precision)

Global precision: 0.5901639344262295
